### Evaluation -- FGSM

In [1]:
import os
import alive_progress
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.models as models
import numpy as np
import lpips
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, Dataset
from scipy.linalg import sqrtm
import torch.nn.functional as F
from torchvision.models.inception import inception_v3
from torchvision.utils import save_image

In [2]:
ITERATION=5000

In [3]:
model = torch.jit.load('../../Models/ResNet152.pt')
model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

RecursiveScriptModule(
  original_name=ResNet
  (conv1): RecursiveScriptModule(original_name=Conv2d)
  (bn1): RecursiveScriptModule(original_name=BatchNorm2d)
  (relu): RecursiveScriptModule(original_name=ReLU)
  (maxpool): RecursiveScriptModule(original_name=MaxPool2d)
  (layer1): RecursiveScriptModule(
    original_name=Sequential
    (0): RecursiveScriptModule(
      original_name=Bottleneck
      (conv1): RecursiveScriptModule(original_name=Conv2d)
      (bn1): RecursiveScriptModule(original_name=BatchNorm2d)
      (conv2): RecursiveScriptModule(original_name=Conv2d)
      (bn2): RecursiveScriptModule(original_name=BatchNorm2d)
      (conv3): RecursiveScriptModule(original_name=Conv2d)
      (bn3): RecursiveScriptModule(original_name=BatchNorm2d)
      (relu): RecursiveScriptModule(original_name=ReLU)
      (downsample): RecursiveScriptModule(
        original_name=Sequential
        (0): RecursiveScriptModule(original_name=Conv2d)
        (1): RecursiveScriptModule(original_name=B

In [4]:
REAL_DATASET="../../Dataset/Imagewoof/train"
ADV_DATASET=f"../../Testing/Images/Generated-Images-{ITERATION}-Encoder"

In [5]:
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [6]:
original_dataset = ImageFolder(REAL_DATASET, transform=transform)
adversarial_dataset = ImageFolder(ADV_DATASET, transform=transform)

In [7]:
original_loader = DataLoader(original_dataset, batch_size=8, shuffle=False)
adversarial_loader = DataLoader(adversarial_dataset, batch_size=8, shuffle=False)

In [8]:
def classify_images(model, dataloader):
    predictions = {}
    with torch.no_grad():
        for i, (images, labels) in enumerate(dataloader):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            for idx, (pred, label) in enumerate(zip(preds.cpu().numpy(), labels.cpu().numpy())):
                predictions[i * dataloader.batch_size + idx] = (pred, label)
    return predictions

In [9]:
%%time
orig_preds = classify_images(model, original_loader)
correctly_classified = {k: v[0] for k, v in orig_preds.items() if v[0] == v[1]}
print(f"Correctly classified original images: {len(correctly_classified)}")

Correctly classified original images: 5514
CPU times: user 46.1 s, sys: 258 ms, total: 46.3 s
Wall time: 46.3 s


In [10]:
%%time
adv_preds = classify_images(model, adversarial_loader)
fooling_count = sum(1 for k, v in correctly_classified.items() if k in adv_preds and adv_preds[k][0] != v)
fooling_rate = fooling_count / len(correctly_classified) if correctly_classified else 0
print(f"Fooling Rate (FR): {fooling_rate:.2f}")

Fooling Rate (FR): 0.88
CPU times: user 41.7 s, sys: 192 ms, total: 41.9 s
Wall time: 41.9 s


In [11]:
print(fooling_count)

4860


## LPIPS

In [12]:
%%time
lpips_fn = lpips.LPIPS(net='alex').to(device)
def compute_lpips(dataloader1, dataloader2):
    lpips_values = []
    for (img1, _), (img2, _) in zip(dataloader1, dataloader2):
        # Ensure both images are moved to the correct device
        img1, img2 = img1.to(device), img2.to(device)
        
        # Compute LPIPS score
        lpips_value = lpips_fn(img1, img2).mean().item()
        lpips_values.append(lpips_value)
    
    # Return the mean LPIPS score
    return np.mean(lpips_values)

# Assuming original_loader and adversarial_loader are defined
lpips_score = compute_lpips(original_loader, adversarial_loader)
print(f"LPIPS Score: {lpips_score:.4f}")

Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]


/opt/miniconda3/envs/superstargan/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/miniconda3/envs/superstargan/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loading model from: /opt/miniconda3/envs/superstargan/lib/python3.12/site-packages/lpips/weights/v0.1/alex.pth


/opt/miniconda3/envs/superstargan/lib/python3.12/site-packages/lpips/lpips.py:107: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.load_state_dict(torch.load(model_path, 

LPIPS Score: 0.5631
CPU times: user 18 s, sys: 662 ms, total: 18.7 s
Wall time: 18.7 s


## FID

In [13]:
class ImageDataset(Dataset):
    def __init__(self, image_dir, transform=None):
        self.image_dir = image_dir
        self.transform = transform
        self.image_paths = [os.path.join(image_dir, img) for img in os.listdir(image_dir) if img.endswith(('jpg', 'png', 'jpeg'))]

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image

class InceptionV3FeatureExtractor(nn.Module):
    def __init__(self):
        super(InceptionV3FeatureExtractor, self).__init__()
        self.inception = models.inception_v3(weights='DEFAULT', transform_input=False)
        self.inception.fc = nn.Identity()

    def forward(self, x):
        return self.inception(x)

In [14]:
def calculate_fid(mu1, sigma1, mu2, sigma2):
    """
    Calculates the FID score between two Gaussian distributions.

    Parameters:
    - mu1: First mean (usually the original one).
    - sigma1: First covariance (usually the original one).
    - mu2: Second mean (usually the adversarial one).
    - sigma2: Second covariance (usually the adversarial one).

    Return:
    - fid: FID score.
    """

    diff = mu1 - mu2
    covmean, _ = sqrtm(sigma1.dot(sigma2), disp=False)
    if np.iscomplexobj(covmean):
        covmean = covmean.real
    fid = diff.dot(diff) + np.trace(sigma1 + sigma2 - 2 * covmean)
    return fid

In [15]:
def get_activations(data_loader, model, device, title):
    """
    Get the activations for an entire Dataloader.

    Parameters:
    - data_loader: A given Dataloader.
    - model: Model to use.
    - device: Device to use (CPU | GPU).

    Return:
    - torch_activations: Activations in a Torch Tensor format.
    """

    model = model.to(device)
    all_activations = []
    with torch.no_grad():
        with alive_progress.alive_bar(len(data_loader), title=f"[ INFO ] {title}", bar='classic', spinner=None) as bar:
            for batch in data_loader:
                if isinstance(batch, (list, tuple)):
                    images = batch[0]
                else:
                    images = batch 
            
                images = images.to(device)
                activations = model(images)
                all_activations.append(activations.cpu())
                bar()
    torch_activations = torch.cat(all_activations, dim=0).numpy()
    torch.cuda.empty_cache()
    return torch_activations


In [16]:
def fid(real_dataset_path, generated_dataset_path):
    """
    Preparation of both real and generated datasets for the FID calculation.

    Parameters:
    - real_dataset_path: Real dataset path.
    - generated_dataset_path: Generated (adversarial) dataset path.
    """

    model = InceptionV3FeatureExtractor().eval()

    transform = transforms.Compose([
        transforms.Resize((299, 299)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    real_dataset = ImageFolder(real_dataset_path, transform=transform)
    generated_dataset = ImageFolder(generated_dataset_path, transform=transform)
    
    real_loader = DataLoader(real_dataset, batch_size=8, shuffle=False, num_workers=4)
    generated_loader = DataLoader(generated_dataset, batch_size=8, shuffle=False, num_workers=4)

    # Calculate activations for real and generated images.
    real_activations = get_activations(real_loader, model, device, title="Calculating the activations of the real images")
    generated_activations = get_activations(generated_loader, model, device, title="Calculating the activations of the generated images")

    # Calculate mean and covariance.
    mu_real = np.mean(real_activations, axis=0)
    sigma_real = np.cov(real_activations, rowvar=False)
    mu_gen = np.mean(generated_activations, axis=0)
    sigma_gen = np.cov(generated_activations, rowvar=False)

    # Compute FID.
    fid_score = calculate_fid(mu_real, sigma_real, mu_gen, sigma_gen)

    return fid_score


In [17]:
%%time
fid_score = fid(real_dataset_path=REAL_DATASET, generated_dataset_path=ADV_DATASET)
print(f"FID Score: {fid_score:.2f}")

[ INFO ] Calculating the activations of the real images [========================================] 1129/1129 [100%] in 1:47.7 (10.49/s) 
[ INFO ] Calculating the activations of the generated images [========================================] 1236/1236 [100%] in 1:36.7 (12.79/s) 
FID Score: 257.39
CPU times: user 4min 19s, sys: 7.43 s, total: 4min 27s
Wall time: 3min 29s
